In [1]:
!git clone https://github.com/naka-tomo/frcnn-from-scratch-with-keras.git

Cloning into 'frcnn-from-scratch-with-keras'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 1193 (delta 4), reused 1 (delta 0), pack-reused 1184
Receiving objects: 100% (1193/1193), 15.93 MiB | 7.55 MiB/s, done.
Resolving deltas: 100% (787/787), done.


In [0]:
%cd frcnn-from-scratch-with-keras/

/content/frcnn-from-scratch-with-keras


In [0]:
# 学習済みモデルをダウンロード
!gdown https://drive.google.com/uc?id=1BL_2ZgTf55vH2q1jvVz0hkhlWYgj-coa
!gdown https://drive.google.com/uc?id=1IgxPP0aI5pxyPHVSM2ZJjN1p9dtE4_64
!mkdir -p models/vgg/
!mv voc.hdf5 models/vgg/

Downloading...
From: https://drive.google.com/uc?id=1BL_2ZgTf55vH2q1jvVz0hkhlWYgj-coa
To: /content/frcnn-from-scratch-with-keras/config.pickle
100% 1.10k/1.10k [00:00<00:00, 527kB/s]
Downloading...
From: https://drive.google.com/uc?id=1IgxPP0aI5pxyPHVSM2ZJjN1p9dtE4_64
To: /content/frcnn-from-scratch-with-keras/voc.hdf5
548MB [00:03, 151MB/s]


In [0]:
%cd frcnn-from-scratch-with-keras/
from __future__ import division
import os
import cv2
import numpy as np
import sys
import pickle
from optparse import OptionParser
import time
import glob
from keras_frcnn import config
from keras import backend as K
from keras.layers import Input
from keras.models import Model
from keras_frcnn import roi_helpers
from keras_frcnn.pascal_voc import pascal_voc_util
from keras_frcnn.pascal_voc_parser import get_data
from keras_frcnn import data_generators

from utils import get_bbox

sys.setrecursionlimit(40000)

"""
parser = OptionParser()

parser.add_option("-p", "--path", dest="test_path", help="Path to test data.")
parser.add_option("-n", "--num_rois", type="int", dest="num_rois",
				help="Number of ROIs per iteration. Higher means more memory use.", default=32)
parser.add_option("--config_filename", dest="config_filename", help=
				"Location to read the metadata related to the training (generated when training).",
				default="config.pickle")
parser.add_option("--network", dest="network", help="Base network to use. Supports vgg or resnet50.", default='resnet50')
parser.add_option("--write", dest="write", help="to write out the image with detections or not.", action='store_true')
parser.add_option("--load", dest="load", help="specify model path.", default=None)
(options, args) = parser.parse_args()
"""

# オプション
class options:
  test_path = "images"
  num_rois = 32
  config_filename = "config.pickle"
  network = "vgg"
  write = True
  load = None

# K.image_dim_ordering()がなくなったため，とりあえず対処
def image_dim_ordering():
  return "tf"

if not options.test_path:   # if filename is not given
	parser.error('Error: path to test data must be specified. Pass --path to command line')

config_output_filename = options.config_filename

with open(config_output_filename, 'rb') as f_in:
	C = pickle.load(f_in)

# we will use resnet. may change to vgg
if options.network == 'vgg':
	C.network = 'vgg16'
	from keras_frcnn import vgg as nn
elif options.network == 'resnet50':
	from keras_frcnn import resnet as nn
	C.network = 'resnet50'
elif options.network == 'vgg19':
	from keras_frcnn import vgg19 as nn
	C.network = 'vgg19'
elif options.network == 'mobilenetv1':
	from keras_frcnn import mobilenetv1 as nn
	C.network = 'mobilenetv1'
elif options.network == 'mobilenetv1_05':
	from keras_frcnn import mobilenetv1_05 as nn
	C.network = 'mobilenetv1_05'
elif options.network == 'mobilenetv1_25':
	from keras_frcnn import mobilenetv1_25 as nn
	C.network = 'mobilenetv1_25'
elif options.network == 'mobilenetv2':
	from keras_frcnn import mobilenetv2 as nn
	C.network = 'mobilenetv2'
else:
	print('Not a valid model')
	raise ValueError

# turn off any data augmentation at test time
C.use_horizontal_flips = False
C.use_vertical_flips = False
C.rot_90 = False

img_path = options.test_path
# Method to transform the coordinates of the bounding box to its original size
def get_real_coordinates(ratio, x1, y1, x2, y2):
	real_x1 = int(round(x1 // ratio))
	real_y1 = int(round(y1 // ratio))
	real_x2 = int(round(x2 // ratio))
	real_y2 = int(round(y2 // ratio))

	return (real_x1, real_y1, real_x2 ,real_y2)
def format_img_size(img, C):
	""" formats the image size based on config """
	img_min_side = float(C.im_size)
	(height,width,_) = img.shape
		
	if width <= height:
		ratio = img_min_side/width
		new_height = int(ratio * height)
		new_width = int(img_min_side)
	else:
		ratio = img_min_side/height
		new_width = int(ratio * width)
		new_height = int(img_min_side)
	img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_CUBIC)
	return img, ratio	

def format_img_channels(img, C):
	""" formats the image channels based on config """
	img = img[:, :, (2, 1, 0)]
	img = img.astype(np.float32)
	img[:, :, 0] -= C.img_channel_mean[0]
	img[:, :, 1] -= C.img_channel_mean[1]
	img[:, :, 2] -= C.img_channel_mean[2]
	img /= C.img_scaling_factor
	img = np.transpose(img, (2, 0, 1))
	img = np.expand_dims(img, axis=0)
	return img

def format_img(img, C):
	""" formats an image for model prediction based on config """
	img, ratio = format_img_size(img, C)
	img = format_img_channels(img, C)
	return img, ratio

class_mapping = C.class_mapping

if 'bg' not in class_mapping:
	class_mapping['bg'] = len(class_mapping)

class_mapping = {v: k for k, v in class_mapping.items()}
print(class_mapping)
class_to_color = {class_mapping[v]: np.random.randint(0, 255, 3) for v in class_mapping}
C.num_rois = int(options.num_rois)

if C.network == 'resnet50':
	num_features = 1024
else:
	# may need to fix this up with your backbone..!
	print("backbone is not resnet50. number of features chosen is 512")
	num_features = 512


if image_dim_ordering() == 'th':
	input_shape_img = (3, None, None)
	input_shape_features = (num_features, None, None)
else:
	input_shape_img = (None, None, 3)
	input_shape_features = (None, None, num_features)


img_input = Input(shape=input_shape_img)
roi_input = Input(shape=(C.num_rois, 4))
feature_map_input = Input(shape=input_shape_features)

# define the base network (resnet here, can be VGG, Inception, etc)
shared_layers = nn.nn_base(img_input, trainable=True)

# define the RPN, built on the base layers
num_anchors = len(C.anchor_box_scales) * len(C.anchor_box_ratios)
rpn_layers = nn.rpn(shared_layers, num_anchors)

classifier = nn.classifier(feature_map_input, roi_input, C.num_rois, nb_classes=len(class_mapping), trainable=True)

model_rpn = Model(img_input, rpn_layers)
#model_classifier_only = Model([feature_map_input, roi_input], classifier)
model_classifier = Model([feature_map_input, roi_input], classifier)

# model loading
if options.load == None:
  print('Loading weights from {}'.format(C.model_path))
  model_rpn.load_weights(C.model_path, by_name=True)
  model_classifier.load_weights(C.model_path, by_name=True)
else:
  print('Loading weights from {}'.format(options.load))
  model_rpn.load_weights(options.load, by_name=True)
  model_classifier.load_weights(options.load, by_name=True)

model_rpn.compile(optimizer='sgd', loss='mse')
model_classifier.compile(optimizer='sgd', loss='mse')

all_imgs = []

classes = {}

bbox_threshold = 0.8

visualise = True

# define pascal
#pascal = pascal_voc_util(options.test_path)

# define dataloader
"""
all_imgs, classes_count, class_mapping = get_data(options.test_path)
val_imgs = [s for s in all_imgs if s['imageset'] == 'test']
if len(val_imgs) == 0:
    print("val images not found. using trainval images for testing.")
    val_imgs = [s for s in all_imgs if s['imageset'] == 'trainval'] # for test purpose
    
print('Num val samples {}'.format(len(val_imgs)))
data_gen_val = data_generators.get_anchor_gt(val_imgs, classes_count, C, nn.get_img_output_length,image_dim_ordering(), mode='val')
"""
#img_pathes = [x["filepath"] for x in val_imgs]
img_pathes = glob.glob( os.path.join( options.test_path, "*" ) )
print(img_pathes)

# define detections
#all_boxes = [[[] for _ in range(len(val_imgs))] for _ in range(20)]

for idx, img_name in enumerate(sorted(img_pathes)):
	if not img_name.lower().endswith(('.bmp', '.jpeg', '.jpg', '.png', '.tif', '.tiff')):
		continue
	print(img_name)
	st = time.time()
	#filepath = os.path.join(img_path,img_name)
  

	img = cv2.imread(img_name)

	X, ratio = format_img(img, C)
	img_scaled = (np.transpose(X[0,:,:,:],(1,2,0)) + 127.5).astype('uint8')

	if image_dim_ordering() == 'tf':
		X = np.transpose(X, (0, 2, 3, 1))

	# get the feature maps and output from the RPN
	[Y1, Y2, F] = model_rpn.predict(X)
	
    # infer roi
	R = roi_helpers.rpn_to_roi(Y1, Y2, C, image_dim_ordering(), overlap_thresh=0.7)
    # get bbox
#	all_dets, bboxes, probs = get_bbox(R, C, model_classifier, class_mapping, F, ratio, bbox_threshold=0.5)
    # convert from (x1,y1,x2,y2) to (x,y,w,h)
	R[:, 2] -= R[:, 0]
	R[:, 3] -= R[:, 1]

	# apply the spatial pyramid pooling to the proposed regions
	bboxes = {}
	probs = {}

	for jk in range(R.shape[0]//C.num_rois + 1):
		ROIs = np.expand_dims(R[C.num_rois*jk:C.num_rois*(jk+1), :], axis=0)
		if ROIs.shape[1] == 0:
			break

		if jk == R.shape[0]//C.num_rois:
			#pad R
			curr_shape = ROIs.shape
			target_shape = (curr_shape[0],C.num_rois,curr_shape[2])
			ROIs_padded = np.zeros(target_shape).astype(ROIs.dtype)
			ROIs_padded[:, :curr_shape[1], :] = ROIs
			ROIs_padded[0, curr_shape[1]:, :] = ROIs[0, 0, :]
			ROIs = ROIs_padded

		[P_cls, P_regr] = model_classifier.predict([F, ROIs])

		for ii in range(P_cls.shape[1]):

			if np.max(P_cls[0, ii, :]) < bbox_threshold: #or np.argmax(P_cls[0, ii, :]) == (P_cls.shape[2] - 1):
#				print("no boxes detected")
				continue
			cls_name = class_mapping[np.argmax(P_cls[0, ii, :])]

			if cls_name not in bboxes:
				bboxes[cls_name] = []
				probs[cls_name] = []

			(x, y, w, h) = ROIs[0, ii, :]

			cls_num = np.argmax(P_cls[0, ii, :])
			try:
				(tx, ty, tw, th) = P_regr[0, ii, 4*cls_num:4*(cls_num+1)]
				tx /= C.classifier_regr_std[0]
				ty /= C.classifier_regr_std[1]
				tw /= C.classifier_regr_std[2]
				th /= C.classifier_regr_std[3]
				x, y, w, h = roi_helpers.apply_regr(x, y, w, h, tx, ty, tw, th)
			except:
				pass
			bboxes[cls_name].append([C.rpn_stride*x, C.rpn_stride*y, C.rpn_stride*(x+w), C.rpn_stride*(y+h)])
			probs[cls_name].append(np.max(P_cls[0, ii, :]))
	all_dets = []
	for key in bboxes:
		bbox = np.array(bboxes[key])

		new_boxes, new_probs = roi_helpers.non_max_suppression_fast(bbox, np.array(probs[key]), overlap_thresh=0.5)
		for jk in range(new_boxes.shape[0]):
			(x1, y1, x2, y2) = new_boxes[jk,:]

			(real_x1, real_y1, real_x2, real_y2) = get_real_coordinates(ratio, x1, y1, x2, y2)

#			cv2.rectangle(img,(real_x1, real_y1), (real_x2, real_y2), (int(class_to_color[key][0]), int(class_to_color[key][1]), int(class_to_color[key][2])),2)
			textLabel = '{}: {}'.format(key,int(100*new_probs[jk]))
			all_dets.append((key,100*new_probs[jk]))
			(retval,baseLine) = cv2.getTextSize(textLabel,cv2.FONT_HERSHEY_COMPLEX,1,1)
#			textOrg = (real_x1, real_y1-0)            

	print('Elapsed time = {}'.format(time.time() - st))
	print("det:", all_dets)
	print("boxes:", bboxes)
    # enable if you want to show pics
	#cv2.imshow('img', img)
	#cv2.waitKey(0)
	if options.write:
           import os
           if not os.path.isdir("results"):
              os.mkdir("results")
           cv2.imwrite('./results/{}.png'.format(idx),img)

[Errno 2] No such file or directory: 'frcnn-from-scratch-with-keras/'
/content/frcnn-from-scratch-with-keras
{0: 'dog', 1: 'cat', 2: 'car', 3: 'person', 4: 'chair', 5: 'bottle', 6: 'diningtable', 7: 'pottedplant', 8: 'bird', 9: 'horse', 10: 'motorbike', 11: 'bus', 12: 'tvmonitor', 13: 'sofa', 14: 'boat', 15: 'cow', 16: 'aeroplane', 17: 'train', 18: 'sheep', 19: 'bicycle', 20: 'bg'}
backbone is not resnet50. number of features chosen is 512
Loading weights from models/vgg/voc.hdf5
['images/85.png', 'images/dog.jpg', 'images/__init__.py']
images/85.png
Elapsed time = 1.1568636894226074
det: [('bg', 97.32570052146912), ('bg', 96.73343896865845), ('bg', 96.72021269798279), ('bg', 96.30354642868042), ('bg', 96.12298607826233), ('bg', 96.01202607154846), ('bg', 96.00116014480591), ('bg', 95.9596335887909), ('bg', 95.9376871585846), ('bg', 95.68564295768738), ('bg', 95.53827047348022), ('bg', 95.33296823501587), ('bg', 95.08544206619263), ('bg', 94.97603178024292), ('bg', 94.89771723747253), 